# Characteristics of Businesses - Group Exercise

In [17]:
import pandas as pd
import requests

tech_variables = 'https://api.census.gov/data/2018/abstcb/variables.html'
owner_variables = 'https://api.census.gov/data/2018/abscbo/variables.html'
characteristics_variables = 'https://api.census.gov/data/2018/abscb/variables.html'
company_summary_variables = 'https://api.census.gov/data/2018/abscs/variables.html'


### Get Tables of Variables

In [30]:
tech_vars = pd.read_html(tech_variables)[0]
owner_vars = pd.read_html(owner_variables)[0]
characteristic_vars = pd.read_html(characteristics_variables)[0]
company_summary_vars = pd.read_html(company_summary_variables)[0]

tech_vars = tech_vars[[_ for _ in tech_vars.columns[:2]]]
owner_vars = owner_vars[[_ for _ in owner_vars.columns[:2]]]
characteristic_vars = characteristic_vars[[_ for _ in characteristic_vars.columns[:2]]]
company_summary_vars = company_summary_vars[[_ for _ in company_summary_vars.columns[:2]]] 

In [34]:
for df in [tech_vars, owner_vars, characteristic_vars, company_summary_vars]:
    print(df.head())
    print("\n")

        Name                                              Label
0        EMP                                Number of employees
1    EMP_PCT                           Percent of employees (%)
2  EMP_PCT_S         Standard error of percent of employees (%)
3      EMP_S  Relative standard error of number of employees...
4  ETH_GROUP                                     Ethnicity code


      Name                         Label
0     CBSA                     Geography
1      for  Census API FIPS 'for' clause
2   GEO_ID    Geographic identifier code
3  GEOCOMP              GEO_ID Component
4       in   Census API FIPS 'in' clause


        Name                                       Label
0    BUSCHAR                Business characteristic code
1       CBSA                                   Geography
2        EMP                         Number of employees
3    EMP_PCT                    Percent of employees (%)
4  EMP_PCT_S  Standard error of percent of employees (%)


        Name          

In [13]:
interest_vars = ",".join(_ for _ in list(set(table.Name.tolist())) if _ not in ['for','in','ucgid','26 variables'])
print(interest_vars)

STATE,NAICS2017,RCPPDEMP,CBSA,NATION,RACE_GROUP,SEX,SUMLEVEL,ETH_GROUP,EMP,RCPPDEMP_S,GEO_ID,PAYANN_S,PAYANN,EMP_S,GEOCOMP,VET_GROUP,YIBSZFI,RCPSZFI,FIRMPDEMP_S,FIRMPDEMP,YEAR,EMPSZFI


In [15]:
tech_path = 'https://api.census.gov/data/2018/absc'
end_point = 'b0?get=&for=us'
complete_path = data_path + interest_vars + end_point
print(complete_path)

https://api.census.gov/data/2019/abscb?get=STATE,NAICS2017,RCPPDEMP,CBSA,NATION,RACE_GROUP,SEX,SUMLEVEL,ETH_GROUP,EMP,RCPPDEMP_S,GEO_ID,PAYANN_S,PAYANN,EMP_S,GEOCOMP,VET_GROUP,YIBSZFI,RCPSZFI,FIRMPDEMP_S,FIRMPDEMP,YEAR,EMPSZFI&for=us
